# Prédiction l'évolution du cours du Bitcoin

Nous y voilà : vous allez devoir travailler maintenant pour proposer votre propre algorithme de descente pour résoudre un problème de classification tout simple. Je vous donne l'historique sur 2020 et 2021 du cours du Bitcoin et vous devez construire un prédicteur capable de détecter si le cours du Bitcoin va partir à la hausse ou à la baisse. 

Vous serez évaluez sur la qualité du travail réalisé et surtout sur les résultats que vous obtiendrez sur des données que vous ne connaissez pas. En fonction de vos résultats par rapport aux autres binômes vous obtiendrez plus ou moins de points. 

Voyons un peu plus les choses en détails.

## Définition du sujet

### Le cours du Bitcoin

Dans les fichiers Cotations2021.csv et Cotations2020.csv vous trouverez l'évolution journalière du cours du Bitcoin en 2021 et en 2020. Ces fichiers contiennent 5 colonnes : 
 - Date : jour de la cotation,
 - Ouverture : valeur en euros du cours du Bitcoin à l'ouverture du marché,
 - Plus Haut : valeur en euros la plus élevée du Bitcoin durant le jour considéré,
 - Plus Bas : valeur en euros la plus basse du Bitcoin durant le jour considéré,
 - Clôture : valeur en euros du cours du Bitcoin à la clôture du marché.

La notion de "cours du Bitcoin" est assez vague : finalement, on se pose vraiment la question de savoir combien vaut le Bitcoin au moment où on doit en acheter ou en vendre... Pour simplifier un peu, nous considérerons que d'un jour $t$ à un jour $(t+1)$ le cours du Bitcoin augmente (diminue) si sa valeur à la clôture au jour $t$ est inférieure (supérieure) à sa valeur à la clôture au jour $(t+1)$. 

Autrement dit, on va regarder la colonne "Clôture" pour décider de l'augmentation ou la baisse du cours du Bitcoin. 

### Enoncé du problème de classification

Le problème que vous avez à résoudre est donc un problème de classification : pour un jour $t$ donné, vous voulez prédire si à $(t+1)$ le cours va augmenter (classe 1) ou diminuer (classe 2). Le cas, marginal, ou le cours reste constant sera assimilé à la classe 2.

Pour réaliser cette prédiction, vous pouvez prendre en compte l'historique du cours sur les $h$ derniers jours. Vous aurez toute liberté de définir cette notion d'historique : c'est d'ailleurs un des facteurs qui pourra influencer la qualité de votre prédicteur. Quelle valeur de $h$ considérer ? Quelle(s) donnée(s) considérer sur les jours précédents : uniquement la valeur de clôture ou ... ? 

### Démarche pour réaliser ce travail

Déjà, vous pouvez fortement vous inspirer de ce que l'on a fait dans la première partie des Travaux Pratiques, ainsi que sur le cours (notamment, le chapitre "Liens avec le Machine Learning", partie sur l'exemple de la société ERAM). 

Au niveau démarche, c'est toujours la même chose : 
1. Définir la notion d'historique : quel vecteur de données allez vous prendre en compte pour l'apprentissage et la prédiction ensuite ? Notons $\vec v$ ce vecteur.
1. Définir le problème d'optimisation à résoudre (fonction $H(\vec v)$ de séparation des données, fonction d'activation (sigmoide), fonction de perte/erreur),
2. Définir le gradient,
3. Créer votre algorithme de descente,

Pour ce dernier point, vous pourrez faire appel à toutes les techniques/éléments vus dans le chapitre "Optimisation continue sans contraintes" quand nous avons parlé des algorithmes de descente.

En plus du code Python ci-dessus, il faudra produire du code de traitement des données et du code pour que je puisse mettre en oeuvre l'évaluation de votre projet.

## Développement de votre algorithme de descente

### Outils conseillés
- La bibliothèque [pandas](https://pandas.pydata.org/docs/user_guide/index.html) est idéale pour l'utilisation d'une base de donnée.
- La bibliothèque [numpy](https://numpy.org/doc/1.18/user/index.html) est parfaite pour tout calcul sur des vecteurs.
- La bibliothèque [matplotlib](https://matplotlib.org/contents.html) est la base pour visualiser des résultats mais accouplé à [seaborn](https://seaborn.pydata.org/index.html) il est possible d'aller plus vite.
- Le bibliothèque [math](https://docs.python.org/fr/3.9/library/math.html) est souvent oubliée tellement elle semble évidente.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import math
import random as rd

### Lecture des données

La première tâche à réaliser va être de développer une fonction qui va lire un fichhier .csv contenant un historique de cotations et créer une base panda. La structure $RawData$ va contenir les données telles que contenues dans le fichier .csv passé en paramètre (selon la syntaxe décrite précédemment). De même, $RawC$ va contenir l'appartenance aux classes 1 (diminution) / 0 (augmentation).

Vous utiliserez les structures $RawData$ et $RawC$ comme vous le souhaiterez par la suite dans vos algorithmes : par contre, ces structures sont à considérer comme des données d'entrée dans toute la suite des TPs.

Autrement dit, pour évaluer vos algorithmes je serais amené à leur donner de nouveaux $RawData$ et $RawC$ contenant des données que vous ne connaissez pas. 

In [2]:
#Input: nom du fichier .csv à lire.
#Nécessite :les lignes du fichier sont classées par dates de cotatation croissantes et les colonnées séparées par ";"
#Output: RawData, contient l'historique des cotations telles que lues dans le .csv
#        RawC, contient pour chaque ligne/date t (sauf la dernière) 1 si le cours baisse à t+1; 0 sinon.
def LireDonnees(NomFichier):
    #Lecture des données dans le fichier csv
    RawData=pd.read_csv(NomFichier, sep=';')
    
    #Construction des classes d'appartenance de chaque ligne de la base: on suppose que les cotations journalières sont
    #classées par ordre chronologique
    RawC=[]
    for index in range(len(RawData)-1):
        value=RawData.iloc[index,4] > RawData.iloc[index+1,4]
        value=1*value
        RawC.append(value)
    return RawData, RawC
    

### Définition des données utilisées pour l'apprentissage/prédiction

Nous allons maintenant définir une méthode qui va préparer une structure de données, nommée $data$, qui va contenir pour chaque ligne un couple :

$\vec v=(\vec x ; c_x)$

où $\vec x$ est le vecteur caractérisant la ligne en cours. Prenons un exemple, que vous adapterez comme vous le souhaitez par la suite. Je décide que pour la ligne $t$, date de cotation $t$, ce qui explique la prédiction $c$ (baisse/augmentation de la valeur de la cotation à $t+1$) c'est le cours du Bitcoin à la clôture sur les $h=3$ derniers jours. Donc, la ligne d'indice 0 dans $data$ va contenir :

$(x_1,x_2,x_3, c_x=0$ ou $1)$ 

avec $x_j$ la valeur de cotation à la clôture le jour $j$.

Créez la fonction CreerData ci-dessous pour avoir par la suite des données plus faciles à manipuler grâce à numpy.

In [3]:
#Input: RawData et RawC ont été créés à partir d'un fichier .csv
#Output: D, contient les points $(\vec x)$. Cela doit être une matrice numpy
#        C, contient la classe d'appartenance $c_x$ de chaque entrée dans data. Cela doit être un vecteur numpy
def CreerData(RawData,RawC):
    h = 3 # On prend les 3 derniers jours
    pas = 1 # en les parcourant de façon linéaire
    variables = ("Ouverture", "Plus Haut", "Plus Bas", "Cloture") # On sélectionne les variables désirées

    nbDates = len(RawData.index)
    nbLignesPerdues = nbDates % h
    nbV = nbDates - nbLignesPerdues

    D = np.ndarray((nbV, h, len(variables)))
    for t in range(nbDates-nbLignesPerdues): # On va instancier chaque v
        for tx in range(0, h, pas): # et leur ajouter chaque x_t
            for xVar in range(len(variables)): # et pour chaque x_t, chaque variable
                D[t][tx][xVar] = RawData[variables[xVar]][t+tx]
    C = np.array(RawC)

    return(D,C)

(RawData,RawC)=LireDonnees("Cotations2020.csv")
(D,C)=CreerData(RawData,RawC)

### Définition du problème d'optimisation à résoudre

On a choisit
$\vec v=(\vec x ; c_x) \in R ^{ h \times 4 }, \ avec \ \vec x=(y_1, y_2, y_3, y_4), \ \vec y$ étant l'ensemble des valeurs des variables sélectionnées

$H(\vec v) = \displaystyle\sum_{i=1}^{h} \vec w_i \cdot \vec x_i + b, \ avec \ h = 3 \ et \ \forall i \in \{1..h\}, \ \vec w_i \in R^4$

D'où $H(\vec v) = \vec w_1 \cdot \vec x_1 + \vec w_2 \cdot \vec x_2 + \vec w_3 \cdot \vec x_3 + b$

On va utiliser la fonction logistique sigmoide pour transformer les valeurs de $H(\vec v)$ en valeurs $0$ ou $1$:

$\mathcal{S}: x \mapsto \frac{1}{1+\exp^{-\lambda x}}$.

Notre classificateur est donc : $\mathcal{S}(H(\vec v))$.

Pour notre problème, on cherche à apprendre sur la base d'apprentissage les valeurs de $\vec w_i \ \forall i \in \{1..h\}$ et $b$.

On choisit la cross-entropy pour mesurer l'erreur de notre problème, c'est à dire la distance entre $\mathcal{S}(H(\vec v))$ et $c_{\vec v}$ : 

$E(\mathcal{P}) = \sum \limits_{\vec v \in \mathcal{P}}-(c_{\vec v}\log_2(\mathcal{S}(H(\vec v)))+(1-c_{\vec v})\log_2(1-\mathcal{S}(H(\vec v))))$.

$\mathcal{P} : \Bigg\{$
$\  Minimiser \ E(P)$
$\ \ s.c.$
$\ \ \vec w_i \in R^4 \ et \ b \in R$
$\Bigg\}$

Commençons par définir le code qui calcule $H(\vec v)$ puis le code de la fonction logistique sigmoid $\cal{S}(x)$. Je suppose que vous allez reprendre les mêmes fonctions que dans le TP introductif, mais si vous voulez changer vous pouvez, bien sûr ! 

In [4]:
# définition de la méthode pour calculer H(v)
def H(weights, constant, point):
    value=np.sum(weights*point)+constant
    return value

Définissons maintenant le code qui calcule $\mathcal{S}(x)$.

In [5]:
# définition de la fonction sigmoide (logistique) et la fonction d'arrondie
def Sigmoid(x, lamda=0.1):
        return 1/(1+np.exp(-lamda*x))


def Step_function(x):
        return 1*(x>=0)

Passons maintenant à la définition de la fonction d'erreur $E(\mathcal{P})$, qui est notre fonction à minimiser, ainsi que du gradient. Je vous laisse les définir !

In [6]:
# définition de la méthode pour calculer l'erreur 
#Input : weights, le vecteur de poids utilisés dans la fonction H,
#        constant, la constante utilisée dans la fonction H
#Output : une valeur numérique, l'erreur commise. 
def Loss_function(weights, constant):
    
    
    return loss_value

In [7]:
# définition du calcul de gradient
#Input : weights, le vecteur de poids utilisés dans la fonction H,
#        constant, la constante utilisée dans la fonction H
#Output : le gradient, un numpy array
def Gradient(weights, constant):
    
    
    return np.array(grad)

### Algorithme de descente

<<Expliquer ici quels sont les caractéristiques/paramètres de votre algorithme de descente>>

<<Justifiez les choix retenus et les choix testés>>

Note: lorsque vous coderez votre algorithme de gradient, il faut que celui-ci affiche dans un graphique l'évolution de la valeur de la fonction d'erreur/loss et l'évolution de la norme du gradient.


In [8]:
# Votre algorithme de gradient qui va travailler à partir des structures numpy D et C
# Input :
# Nécessite :
# Output : weights, un numpy array contenant les poids utilisés dans la fonction de séparation H 
#          constant, un numérique contenant la constante utilisée dans la fonction de séparation H
#          nbstep, nombre d'itérations mises par l'algorithme avant de s'arrêter

# ATTENTION: j'ai laissé dans l'algorithme vide ci-dessous des lignes de code permettant l'affichage de la fonction de perte/loss
#            et la norme du gradient. Gardez ce code qui vous aidera à voir si votre algorithme converge ou pas.

def Algorithme_Gradient():
    
   #Initialisations pour les graphiques terminaux: à conserver
    ## Produire deux graphes cote à cote 2 subplots
    fig, ax = plt.subplots(figsize=(10, 5) , nrows=1, ncols=1)
    ## Pour visualiser les evolutions en fonction du step
    itersteps = []
    ## Pour visualiser l'evolution de la norme du gradient
    iterGap = []
    ## Pour visualiser l'évolution de l'erreur
    iterError = []
    

    

    # Affichage final : à conserver
    ax.plot(itersteps,iterGap, label='Norme du gradient')
    ax.plot(itersteps,iterError, label='Erreur')
    ax.set_xlabel('step')
    ax.legend()
    plt.show()
    
    return(weights,constant,nbstep)


## Phase d'apprentissage 

Allez, on se lance ! Exécutez votre algorithme d'apprentissage pour voir ce qu'il donne. Je vous met ci-dessous un code générique que vous pouvez légèrement adapter. Néanmoins, faîtes bien attention que : 
* La fonction LireDonnees() doit être appelée pour lire des données brutes dans un fichier .csv qui est votre base d'apprentissage,
* La fonction CréerData() doit être appelée pour créer les structures $D$ et $C$ qui sont utilisées par votre algorithme de gradient,
* Aprés vous appelez l'algorithme de gradient que vous avez vous-même définis.

In [9]:
(RawData,RawC)=LireDonnees(<<Nom_fichier_csv>>)
(D,C)=CreerData(RawData,RawC)
(weights,constant)=Algorithme_Gradient()

SyntaxError: invalid syntax (1840706473.py, line 1)

## Construction du prédicteur

Il faut maintenant construire votre prédicteur : je vous propose le code suivant.

In [ ]:
# Input: weights, les poids appris pendant l'apprentissage
#       constant, la constante apprise pendant l'apprentissage
#       v, le vecteur à partir duquel on veut prédire
# Nécessite : que l'apprentissage ait été fait, i.e. on dispose de weights et constant
# Output : le résultat de la prédiction
def Prediction(weights, constant, v):
    if Sigmoid(H(weights, constant, v))>0.5:
            return 1
    else: 
            return 0

Généralisons un peu ce prédicteur pour l'évaluer sur une base de vecteurs $\vec v$

In [ ]:
# Input: weights, les poids appris pendant l'apprentissage
#       constant, la constante apprise pendant l'apprentissage
#       D, une base de données de vecteurs à partir duquel on veut prédire
# Nécessite : que l'apprentissage ait été fait, i.e. on dispose de weights et constant
# Output : Cpred, le vecteur des prédictions associées

def PredictionsOnBase(weigths,constant,D):
    Cpred=[]
    for index in range(len(D)):
        Cpred.append(Prediction(weights,constant,D[index:index+1,:]))
    return Cpred

## Evaluation du prédicteur sur la base d'apprentissage

Le code ci-dessous va vous permettre d'évaluer la qualité de votre prédicteur sur la base d'apprentissage, autrement dit:
- Calcul du pourcentage de bonnes classifications,
- Matrice de confusion : vous donnera la répartition des bonnes/mauvaises classifications.

In [ ]:
# Pour exécuter ce code vous devez avoir au préalable exécuté l'apprentissage => on doit disposer des weights et constant

Cpred=PredictionsOnBase(weights,constant,D)
GoodPred=0
VraiPositifs=0
FauxPositifs=0
VraiNegatifs=0
FauxNegatifs=0
NumberPositif=0
for index in range(len(C)):
    if (C[index]==1): 
        NumberPositif +=1
    if (C[index]==Cpred[index]):
        GoodPred+=1
    if (C[index]==1):
        if (Cpred[index]==1):
            VraiPositifs+=1
        else:
            FauxNegatifs+=1
    if (C[index]==0):
        if (Cpred[index]==1):
            FauxPositifs+=1
        else:
            VraiNegatifs+=1
print("Taux de bonnes classifications : ",100.0*GoodPred/len(C),"%")
print("Quand il fallait prédire 1...")
print(100.0*VraiPositifs/NumberPositif,"% du temps le prédicteur prédit 1")
print(100.0*FauxNegatifs/NumberPositif,"% du temps le prédicteur prédit 0")
print("Quand il fallait prédire 0...")
print(100.0*VraiNegatifs/(len(C)-NumberPositif),"% du temps le prédicteur prédit 0")
print(100.0*FauxPositifs/(len(C)-NumberPositif),"% du temps le prédicteur prédit 1")